In [3]:
import IPython.display as display
import sys
sys.path.append('../algorithms/py')

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
import optuna
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, roc_auc_score

In [6]:
import rutils as rt

# Data Loading

In [7]:
edata_df = pd.read_csv('../mutual-data/merged_edata.tsv', sep='\t', index_col=0)
mdata_df = pd.read_csv('../mutual-data/merged_mdata.tsv', sep='\t', index_col=0)
print( np.all(edata_df.columns == mdata_df.index) )

with open('../wgcna/common_genes.txt') as f:
	filter_genes = f.readlines()

gene_names = np.array([l for line in filter_genes if (l:=line.strip()) != ''])
print(f'{gene_names.size=}')

edata_df = edata_df.loc[gene_names,]
X = edata_df.T.to_numpy()
y = np.int_(mdata_df.condition == 'SC')
np.bincount(y)

True
gene_names.size=101


array([33, 14])

# START FROM HERE

In [8]:
def find_out(X, y, model, n_boot=200, seed=None):
  rnd = np.random.default_rng(seed)
  coef_arr = []
  score_arr = []
  for i in range(n_boot):
      X_boot, X_oob, y_boot, y_oob = rt.bootstrap_resample(X, y, rnd_engine=rnd)
      scaler = StandardScaler()
      X_boot_scaled = scaler.fit_transform(X_boot)
      X_oob_scaled = scaler.transform(X_oob)

      model.fit(X_boot_scaled, y_boot)
      coef_arr.append(model.coef_[0])
      # rt.iterLog(i)

      # y_oob_pred_proba = model.predict_proba(X_oob_std)[:, 1]
      score_arr.append(
          rt.pr_auc(y_oob, model.predict_proba(X_oob_scaled)[:, 1])
      )

  coef_arr = np.array(coef_arr)
  score_arr = np.array(score_arr)
  return coef_arr, score_arr

## define objective function

In [9]:
def objective(trial):
  C_val = trial.suggest_float('C', 0.01, 100, log=True)
  lr_val = trial.suggest_float('lr_ratio', 0.1, 0.9)
  threshold = trial.suggest_float('thrs', 0.2, 0.8)

  lr_model = LogisticRegression(
        penalty='elasticnet', solver='saga',
        class_weight='balanced',
        max_iter=10000, random_state=791176,
        C=C_val, l1_ratio=lr_val
    )

  coef_arr, score_arr = find_out(X, y, lr_model, n_boot=400, seed=209741)


  selected_genes = np.mean(np.abs(coef_arr) > 1e-6, axis=0) >= threshold

  if np.sum(selected_genes) == 0:
        return 0

  oob_score = rt.evaluateBootstrap(
        X, y, lr_model, selected_genes, n_boot=400,
        use_proba=True, metric=rt.pr_auc, seed=522863, boot_score=False
    )

  return oob_score.mean()

In [10]:
study = optuna.create_study(
    study_name = 'LogisticRegression',
    direction='maximize',
    sampler=optuna.samplers.TPESampler(seed=372940)
)

[I 2025-07-26 20:49:58,977] A new study created in memory with name: LogisticRegression


In [11]:
study.optimize(objective, n_trials=100)

[I 2025-07-26 20:50:47,206] Trial 0 finished with value: 0.8348502620249605 and parameters: {'C': 7.771705089424058, 'lr_ratio': 0.7783057814789166, 'thrs': 0.6320751177245301}. Best is trial 0 with value: 0.8348502620249605.
[I 2025-07-26 20:51:47,677] Trial 1 finished with value: 0.7297086898726078 and parameters: {'C': 24.979325234220823, 'lr_ratio': 0.6697665566043789, 'thrs': 0.2183253221856687}. Best is trial 0 with value: 0.8348502620249605.
[I 2025-07-26 20:52:43,544] Trial 2 finished with value: 0.7304627233904849 and parameters: {'C': 55.451892812748916, 'lr_ratio': 0.6820178883035428, 'thrs': 0.5930718611494435}. Best is trial 0 with value: 0.8348502620249605.
[I 2025-07-26 20:53:22,307] Trial 3 finished with value: 0.8407754315229163 and parameters: {'C': 6.764659585661619, 'lr_ratio': 0.7606126371809446, 'thrs': 0.77525285893477}. Best is trial 3 with value: 0.8407754315229163.
[I 2025-07-26 20:54:09,516] Trial 4 finished with value: 0.846349723564403 and parameters: {'C':

In [12]:
df = study.trials_dataframe()
df.head()

,number,value,datetime_start,datetime_complete,duration,params_C,params_lr_ratio,params_thrs,state
0,0,0.834850,2025-07-26 20:50:04.992969,2025-07-26 20:50:47.205981,0 days 00:00:42.213012,7.771705,0.778306,0.632075,COMPLETE
1,1,0.729709,2025-07-26 20:50:47.207674,2025-07-26 20:51:47.677555,0 days 00:01:00.469881,24.979325,0.669767,0.218325,COMPLETE
2,2,0.730463,2025-07-26 20:51:47.679088,2025-07-26 20:52:43.544670,0 days 00:00:55.865582,55.451893,0.682018,0.593072,COMPLETE
3,3,0.840775,2025-07-26 20:52:43.548560,2025-07-26 20:53:22.307490,0 days 00:00:38.758930,6.764660,0.760613,0.775253,COMPLETE
4,4,0.846350,2025-07-26 20:53:22.308786,2025-07-26 20:54:09.516031,0 days 00:00:47.207245,12.690178,0.873082,0.738753,COMPLETE


In [18]:
import plotly

In [23]:
# {'C': 0.4935835883659901,
#  'lr_ratio': 0.3941342547707363,
#  'thrs': 0.7587094413433819}
study.best_params

{'C': 0.4935835883659897,
 'lr_ratio': 0.3941342547707365,
 'thrs': 0.7587094413433819}

In [24]:
lr_model2 = LogisticRegression(
         C=study.best_params['C'], l1_ratio=study.best_params['lr_ratio'],
         penalty='elasticnet', solver='saga',
         class_weight='balanced',
         max_iter=10000, random_state=42
     )

In [25]:
coef_arr, score_arr = find_out(X, y, lr_model2, n_boot=400, seed=4356)

In [26]:
indices = np.mean(coef_arr != 0, axis=0) >= study.best_params['thrs']
gene_names[indices]

array(['ALDH3A1', 'PIR', 'CLTB', 'TXNRD1', 'NUMA1', 'ETFB', 'HRAS'],
      dtype='<U10')

In [27]:
boot, oob = rt.evaluateBootstrap(
    X, y, lr_model2, indices, n_boot=1000, seed=231, boot_score=True,
    use_proba=True, metric=rt.pr_auc
)
print(boot.mean(), oob.mean())
oob.var()

0.9348841853135925 0.8731256774201284


0.010289080949169993

In [28]:
train, test = rt.evaluateLOO(
    X, y, lr_model2, indices, metric=f1_score, use_proba=False, train_score=True, verbose=1
)
train.mean(), test

[[30  3]
 [ 2 12]]


(0.8664127915778683, 0.8275862068965517)

In [29]:
coef_arr1, _ = find_out(X[:,indices], y, lr_model2, n_boot=400, seed=231)
print(np.mean(coef_arr1, 0))
print(np.median(coef_arr1, 0))

[-0.58868392 -0.29782474  0.72941471 -0.1599811   0.75908445  0.31878672
  0.50324021]
[-0.60239973 -0.25036619  0.7456899  -0.09966627  0.75034951  0.29781366
  0.47492911]


In [30]:
rt.writeln(gene_names[indices])

ALDH3A1
PIR
CLTB
TXNRD1
NUMA1
ETFB
HRAS
